In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

## Start by scrapping the main page

In [12]:
# regulate frequency of request
import time

## Parsing the main page with BeautifulSoup

### Find number of rental "listing pages" to scrap

In [13]:
# boroughs of NY
boroughs = ['manhattan-ny','queens-ny','brooklyn-ny','staten-island-ny','bronx-ny']
boroughs_max_page = {'manhattan-ny':0,'queens-ny':0,'brooklyn-ny':0,'staten-island-ny':0,'bronx-ny':0}
boroughs_listing_pages = {'manhattan-ny':{},'queens-ny':{},'brooklyn-ny':{},'staten-island-ny':{},'bronx-ny':{}}

# send user agent to avoid bot check
headers = {'User-Agent': 'User'}

# loop over borooughs of NY
for br in boroughs:
    
    # request page
    page_main = requests.get('https://www.apartments.com/%s/'%br, headers=headers)
    
    # pause to regulate request frequency
    time.sleep(1)

    # create soup of the main page
    soup_main = BeautifulSoup(page_main.content,'html.parser')

    # for testing
    #print(soup_main.prettify())

    # find the total number of apartment "listing pages" to scrap
    max_list_pages = 0

    # find tag that links to "listing pages"
    for tag in soup_main.find_all('a'):
        # selecting tags containing page numbers
        if 'data-page' in tag.attrs:
            if not 'class' in tag.attrs:
                # found the page numbers to scrap for this br
                boroughs_max_page[br] = max(int(tag['data-page']),max_list_pages)

# for testing            
print(boroughs_max_page)

{'manhattan-ny': 28, 'queens-ny': 28, 'brooklyn-ny': 28, 'staten-island-ny': 22, 'bronx-ny': 28}


## Scrapping "listing pages"

In [14]:
# for testing only
#br = 'queens-ny'
#boroughs_max_page[br] = 2

# loop over boroughs
for br in boroughs:
#if True:

    # loop over page number
    for i in range(boroughs_max_page[br]):
        
        # set page number to avoid confusion
        page_number = i+1
        
        print("Loop over page %s of %s" %(page_number,br))
        
        # request list page
        page_listing = requests.get('https://www.apartments.com/%s/%s' %(br,page_number), headers=headers)
        
        # pause to regulate request frequency
        time.sleep(1)
       
        # parse listing page with BeautifulSoup
        soup_listing = BeautifulSoup(page_listing.content,'html.parser')
       
        # for testing
        #print(soup_listing.prettify())

        for tag in soup_listing.findAll("a",{"class":"placardTitle"}):
            
            # add title and the link to each property
            if not tag['title'] in boroughs_listing_pages[br]:
                boroughs_listing_pages[br][tag['title']] = tag['href']
            

print("collected links to each apartment pages")

Loop over page 1 of manhattan-ny
Loop over page 2 of manhattan-ny
Loop over page 3 of manhattan-ny
Loop over page 4 of manhattan-ny
Loop over page 5 of manhattan-ny
Loop over page 6 of manhattan-ny
Loop over page 7 of manhattan-ny
Loop over page 8 of manhattan-ny
Loop over page 9 of manhattan-ny
Loop over page 10 of manhattan-ny
Loop over page 11 of manhattan-ny
Loop over page 12 of manhattan-ny
Loop over page 13 of manhattan-ny
Loop over page 14 of manhattan-ny
Loop over page 15 of manhattan-ny
Loop over page 16 of manhattan-ny
Loop over page 17 of manhattan-ny
Loop over page 18 of manhattan-ny
Loop over page 19 of manhattan-ny
Loop over page 20 of manhattan-ny
Loop over page 21 of manhattan-ny
Loop over page 22 of manhattan-ny
Loop over page 23 of manhattan-ny
Loop over page 24 of manhattan-ny
Loop over page 25 of manhattan-ny
Loop over page 26 of manhattan-ny
Loop over page 27 of manhattan-ny
Loop over page 28 of manhattan-ny
Loop over page 1 of queens-ny
Loop over page 2 of queens-

In [15]:
# for dibugging
#for title, url in boroughs_listing_pages[br].items():
#    print(title)
#    print(url)
for br in boroughs:
    print(len(boroughs_listing_pages[br]))


691
668
700
539
658


##  scrap rental listing, store them in DF

In [16]:
# define lists to hold contents before creating DataFrame
list_rental_title = []
list_street_address = []
list_city = []
list_state = []
list_postal_code = []
list_rating = []
list_amenty = []
list_pet_policy = []
list_property_info = []
list_school = []
list_bedrooms = []
list_bathrooms = []
list_rent = []
list_deposit = []
list_unit = []
list_sqft = []
list_name = []
list_leaseLength = []
list_borough = []

# loop over boroughs again to scrap rental listing
for br in boroughs:
    
    # loop over rental listing
    for title, url in boroughs_listing_pages[br].items():
        
        print("boroughs: %s, processing apartment: %s" %(br,title))
        
        #=======================================
        # request and parse each complex
        #=======================================
        
        # get url from the list
        rental_url = url
        
        # request list page
        page_rent = requests.get('%s' %rental_url, headers=headers)
        
        # pause to regulate request frequency
        time.sleep(0.2)
        
        # parse listing page with BeautifulSoup
        soup_rent = BeautifulSoup(page_rent.content,'html.parser')
        
        # for testing
        #print(soup_rent.prettify())
        
        #=======================================
        # find complex's property
        #=======================================
        
        # get rental title
        rental_title = title
        
        # street address
        if soup_rent.findAll("span",{"itemprop":"streetAddress"}):
            street_address = soup_rent.findAll("span",{"itemprop":"streetAddress"})[0].text
        else:
            street_address = ""
        
        # city
        if soup_rent.findAll("span",{"itemprop":"addressLocality"}):
            city = soup_rent.findAll("span",{"itemprop":"addressLocality"})[0].text
        else:
            city = ""
        
        # state
        if soup_rent.findAll("span",{"itemprop":"addressRegion"}):
            state = soup_rent.findAll("span",{"itemprop":"addressRegion"})[0].text
        else:
            state = ""
        
        # postal code (zip code)
        if soup_rent.findAll("span",{"itemprop":"postalCode"}):
            postal_code = soup_rent.findAll("span",{"itemprop":"postalCode"})[0].text
        else:
            postal_code = ""
        
        # apartment rating
        if 'title' in soup_rent.findAll("div",{"class":"rating"})[0].attrs:
            rating = soup_rent.findAll("div",{"class":"rating"})[0]['title']
        else:
            rating = ""
        
        # list of amenties
        amenty_temp = []
        if soup_rent.findAll("section",{"class":"printPropertySection"}):
            for tag_amenty in soup_rent.findAll("section",{"class":"printPropertySection"})[0].findAll("li"):
                amenty_temp.append(tag_amenty.text)
            
        # add amenty to row
        amenty = amenty_temp
        
        # pet policy
        pet_policy_temp = []
        for tags_pet in soup_rent.findAll("div",{"class":"petPolicyDetails"}):
            pet_policy_temp.append(tags_pet.text)
            
        # add pet policy to row
        pet_policy = pet_policy_temp
            
        # additional information such as built date, complex size    
        if soup_rent.findAll("div",{"class":"specList propertyFeatures js-spec"}):
            property_info = soup_rent.findAll("div",{"class":"specList propertyFeatures js-spec"})[0].text
        else:
            property_info = ""
        
        # nearby school information
        school_temp = []
        for tag_school in soup_rent.findAll("div",{"class":"schoolCard"}):
            # get name, number of students, rating
            school_name = tag_school.findAll("p",{"class":"schoolType"})[0].text
            school_number_student = tag_school.findAll("p",{"class":"numberOfStudents"})[0].text
            school_rating = tag_school.findAll("i")[0]['class'][0]
            
            # add it to list of schools as a dictionary
            school_temp.append({"school_name":school_name, "school_number_student":school_number_student,"school_rating":school_rating})
            
        # add school to row
        school = school_temp

        # loop over available units in this apartment
        for tag_unit in soup_rent.findAll("tr",{"class":"rentalGridRow"}):
        
            #=======================================
            # find unit property
            #=======================================
          
            # append complex's properties
            list_rental_title.append(rental_title)
            list_street_address.append(street_address)
            list_city.append(city)
            list_state.append(state)
            list_postal_code.append(postal_code)
            list_rating.append(rating)
            list_amenty.append(amenty)
            list_pet_policy.append(pet_policy)
            list_property_info.append(property_info)
            list_school.append(school)
            list_borough.append(br)
            
            # number of bedrooms in this apartment
            if tag_unit.findAll("td",{"class":"beds"}):
                list_bedrooms.append(tag_unit.findAll("td",{"class":"beds"})[0].findAll("span",{"class":"shortText"})[0].text.strip())
            else:
                list_bedrooms.append("")
            
            # number of bathroom in this apartment
            if tag_unit.findAll("td",{"class":"baths"}):
                list_bathrooms.append(tag_unit.findAll("td",{"class":"baths"})[0].findAll("span",{"class":"shortText"})[0].text.strip())
            else:
                list_bathrooms.append("")
            
            # rent per month
            if tag_unit.findAll("td",{"class":"rent"}):
                list_rent.append(tag_unit.findAll("td",{"class":"rent"})[0].text.strip())
            else:
                list_rent.append("")
            
            # deposit
            if tag_unit.findAll("td",{"class":"deposite"}):
                list_deposit.append(tag_unit.findAll("td",{"class":"deposit"})[0].text.strip())
            else:
                list_deposit.append("")
            
            # unit
            if tag_unit.findAll("td",{"class":"unit"}):
                list_unit.append(tag_unit.findAll("td",{"class":"unit"})[0].text.strip())
            else:
                list_unit.append("")
        
            # sqft
            if tag_unit.findAll("td",{"class":"sqft"}):
                list_sqft.append(tag_unit.findAll("td",{"class":"sqft"})[0].text.strip())
            else:
                list_sqft.append("")
        
            # name
            if tag_unit.findAll("td",{"class":"name"}):
                list_name.append(tag_unit.findAll("td",{"class":"name"})[0].text.strip())
            else:
                list_name.append("")
                
            # lease length
            if tag_unit.findAll("td",{"class":"leaseLength"}):
                list_leaseLength.append(tag_unit.findAll("td",{"class":"leaseLength"})[0].text.strip())
            else:
                list_leaseLength.append("")
            
                    


boroughs: manhattan-ny, processing apartment: Avalon Morningside Park, New York, NY
boroughs: manhattan-ny, processing apartment: Avalon Midtown West, New York, NY
boroughs: manhattan-ny, processing apartment: Avalon Clinton North, New York, NY
boroughs: manhattan-ny, processing apartment: The Eugene, New York, NY
boroughs: manhattan-ny, processing apartment: Dorchester Towers, New York, NY
boroughs: manhattan-ny, processing apartment: Normandie Court, New York, NY
boroughs: manhattan-ny, processing apartment: 21 WEA, New York, NY
boroughs: manhattan-ny, processing apartment: EOS, New York, NY
boroughs: manhattan-ny, processing apartment: VIA 57 WEST, New York, NY
boroughs: manhattan-ny, processing apartment: 555TEN, New York, NY
boroughs: manhattan-ny, processing apartment: 212 West 20th Street, New York, NY
boroughs: manhattan-ny, processing apartment: Murray Hill Marquis, New York, NY
boroughs: manhattan-ny, processing apartment: Stuyvesant Town, New York, NY
boroughs: manhattan-ny,

boroughs: manhattan-ny, processing apartment: 338 W 45th St Unit 112, New York, NY
boroughs: manhattan-ny, processing apartment: 67 Pine St Unit 5106, New York, NY
boroughs: manhattan-ny, processing apartment: 529 W 179th St Unit 4B, New York, NY
boroughs: manhattan-ny, processing apartment: 195 Stanton St, New York, NY
boroughs: manhattan-ny, processing apartment: 529 Broome St, New York, NY
boroughs: manhattan-ny, processing apartment: 973 Amsterdam Ave, New York, NY
boroughs: manhattan-ny, processing apartment: 512 E 83rd St, New York, NY
boroughs: manhattan-ny, processing apartment: 540 W 51st St, New York, NY
boroughs: manhattan-ny, processing apartment: 97 Worth St, New York, NY
boroughs: manhattan-ny, processing apartment: 604 W 136th St, New York, NY
boroughs: manhattan-ny, processing apartment: 535 W 43rd St, New York, NY
boroughs: manhattan-ny, processing apartment: 546 Main St Unit 43, New York, NY
boroughs: manhattan-ny, processing apartment: 510 Main St Unit 102, New York,

boroughs: manhattan-ny, processing apartment: 200 W 88th St Unit 2C, New York, NY
boroughs: manhattan-ny, processing apartment: 184 E 93rd St Unit 4, New York, NY
boroughs: manhattan-ny, processing apartment: 337 E 90th St Unit 5D, New York, NY
boroughs: manhattan-ny, processing apartment: 324 W 84th St Unit 34, New York, NY
boroughs: manhattan-ny, processing apartment: 113 Sullivan St Unit 402, New York, NY
boroughs: manhattan-ny, processing apartment: 236 E 36th St Unit 1G, New York, NY
boroughs: manhattan-ny, processing apartment: 3 Haven Plaza Unit 5, New York, NY
boroughs: manhattan-ny, processing apartment: 63 Pitt St Unit 5, New York, NY
boroughs: manhattan-ny, processing apartment: 29 Clinton St, New York, NY
boroughs: manhattan-ny, processing apartment: 57 Pitt St Unit 5, New York, NY
boroughs: manhattan-ny, processing apartment: 1976 1st Avenue Unit 2E, New York, NY
boroughs: manhattan-ny, processing apartment: 100 Pinehurst Ave, New York, NY
boroughs: manhattan-ny, processin

boroughs: manhattan-ny, processing apartment: 603 W 47th St, New York, NY
boroughs: manhattan-ny, processing apartment: 312 11th Ave, New York, NY
boroughs: manhattan-ny, processing apartment: 1381 Riverside Dr, New York, NY
boroughs: manhattan-ny, processing apartment: 295 Park Ave S, New York, NY
boroughs: manhattan-ny, processing apartment: 145 4th Ave Unit 6A, New York, NY
boroughs: manhattan-ny, processing apartment: 77 W 24th St Unit 21, New York, NY
boroughs: manhattan-ny, processing apartment: 345 E 61st St Unit 40, New York, NY
boroughs: manhattan-ny, processing apartment: 375 Central Park West Unit 70, New York, NY
boroughs: manhattan-ny, processing apartment: 340 W 87th St, New York, NY
boroughs: manhattan-ny, processing apartment: 317 W 99th St, New York, NY
boroughs: manhattan-ny, processing apartment: 132 E 61st St, New York, NY
boroughs: manhattan-ny, processing apartment: 376 South End Ave Unit 27K, New York, NY
boroughs: manhattan-ny, processing apartment: 90 West St U

boroughs: manhattan-ny, processing apartment: 417 E 65th St Unit 5, New York, NY
boroughs: manhattan-ny, processing apartment: 853 7th Ave Unit 8B, New York, NY
boroughs: manhattan-ny, processing apartment: 315 W 21st St Unit 1G, New York, NY
boroughs: manhattan-ny, processing apartment: 415 E 72nd St Unit 6F, New York, NY
boroughs: manhattan-ny, processing apartment: 102 W 80th St Unit 47, New York, NY
boroughs: manhattan-ny, processing apartment: 11 E 80th St Unit 5B, New York, NY
boroughs: manhattan-ny, processing apartment: 425 5th Ave Unit 47C, New York, NY
boroughs: manhattan-ny, processing apartment: 310 E 2nd St, New York, NY
boroughs: manhattan-ny, processing apartment: 90 West St, New York, NY
boroughs: manhattan-ny, processing apartment: 21 Exchange Pl, New York, NY
boroughs: manhattan-ny, processing apartment: 125 W 31st St Unit 14J, New York, NY
boroughs: manhattan-ny, processing apartment: 144 W 86th St Unit 17B, New York, NY
boroughs: manhattan-ny, processing apartment: 

...

boroughs: bronx-ny, processing apartment: 1845 Mohegan Ave Unit 14, Bronx, NY
boroughs: bronx-ny, processing apartment: 3582 Netherland Ave Unit 1R, Bronx, NY
boroughs: bronx-ny, processing apartment: 3056 Godwin Terrace Unit 8, Bronx, NY
boroughs: bronx-ny, processing apartment: 392 E 153rd St Unit 2F, Bronx, NY
boroughs: bronx-ny, processing apartment: 317 Alexander Ave Unit 4F, Bronx, NY
boroughs: bronx-ny, processing apartment: 3624 Oxford Ave Unit 4B, Bronx, NY
boroughs: bronx-ny, processing apartment: 1724 University Ave Unit 2H, Bronx, NY
boroughs: bronx-ny, processing apartment: 626 E 169th St Unit 4D, Bronx, NY
boroughs: bronx-ny, processing apartment: 1268 Sheridan Ave Unit 6H, Bronx, NY
boroughs: bronx-ny, processing apartment: 532 W 236th St Unit 5B, Bronx, NY
boroughs: bronx-ny, processing apartment: 2468 Belmont Ave Unit 2, Bronx, NY
boroughs: bronx-ny, processing apartment: 952 Underhill Ave Unit 5D, Bronx, NY
boroughs: bronx-ny, processing apartment: 1027 Walton Ave Uni

boroughs: bronx-ny, processing apartment: 1551 Williamsbridge Rd, Bronx, NY
boroughs: bronx-ny, processing apartment: 307 E 239th St, Bronx, NY
boroughs: bronx-ny, processing apartment: 3210 Kingsbridge Ave Unit 5A, Bronx, NY
boroughs: bronx-ny, processing apartment: 221 W 233rd St Unit 1J, Bronx, NY
boroughs: bronx-ny, processing apartment: 222 W 233rd St Unit E, Bronx, NY
boroughs: bronx-ny, processing apartment: 3214 Kingsbridge Ave Unit 5A, Bronx, NY
boroughs: bronx-ny, processing apartment: 6215 Broadway Unit 2E, Bronx, NY
boroughs: bronx-ny, processing apartment: 221 W 233rd St Unit 2J, Bronx, NY
boroughs: bronx-ny, processing apartment: 3207 Kingsbridge Ave Unit 2F, Bronx, NY
boroughs: bronx-ny, processing apartment: 222 W 233rd St Unit 2L, Bronx, NY
boroughs: bronx-ny, processing apartment: 6215 Broadway Unit 2D, Bronx, NY
boroughs: bronx-ny, processing apartment: 221 W 233rd St Unit B, Bronx, NY
boroughs: bronx-ny, processing apartment: 3217 Kingsbridge Ave Unit 5E, Bronx, NY


boroughs: bronx-ny, processing apartment: 831 Bartholdi St, Bronx, NY
boroughs: bronx-ny, processing apartment: 1364 Bronx River Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 1704 Morris Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 1031 Walton Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 1827 Walton Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 390 Melrose Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 1511 Sheridan Ave Unit C23, Bronx, NY
boroughs: bronx-ny, processing apartment: 1600 Netherland Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 1727 Mahan Ave Unit #2, Bronx, NY
boroughs: bronx-ny, processing apartment: 280 Concord Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 1727 Mahan Ave Unit 1, Bronx, NY
boroughs: bronx-ny, processing apartment: 2358 Grand Concourse, Bronx, NY
boroughs: bronx-ny, processing apartment: 701 Neill Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 2685 Creston Ave Unit 3H

boroughs: bronx-ny, processing apartment: 674 St Ann&#39;s Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 1702 Melville St Unit 2, Bronx, NY
boroughs: bronx-ny, processing apartment: 1105 Morrison Ave Unit 2E, Bronx, NY
boroughs: bronx-ny, processing apartment: 652 E 230th St Unit 1, Bronx, NY
boroughs: bronx-ny, processing apartment: 2275 Grand Ave Unit 1BR, Bronx, NY
boroughs: bronx-ny, processing apartment: 1970 E Tremont Ave Unit 9, Bronx, NY
boroughs: bronx-ny, processing apartment: 6229 Broadway Unit 2C, Bronx, NY
boroughs: bronx-ny, processing apartment: 6225 Broadway Unit B, Bronx, NY
boroughs: bronx-ny, processing apartment: 2105 Ryer Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 2100 Ryer Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 622 E 169th St, Bronx, NY
boroughs: bronx-ny, processing apartment: 626 Manida St, Bronx, NY
boroughs: bronx-ny, processing apartment: 3660 Waldo Ave Unit 2J, Bronx, NY
boroughs: bronx-ny, processing apartment: 112

boroughs: bronx-ny, processing apartment: 1633 Macombs Rd Unit 4A, Bronx, NY
boroughs: bronx-ny, processing apartment: 4499 Henry Hudson Pkwy Unit 6B, Bronx, NY
boroughs: bronx-ny, processing apartment: 85 Strong St Unit A43, Bronx, NY
boroughs: bronx-ny, processing apartment: 2391 Davidson Ave Unit C2, Bronx, NY
boroughs: bronx-ny, processing apartment: 4499 Henry Hudson Pkwy W Unit #5E, Bronx, NY
boroughs: bronx-ny, processing apartment: 144 Bruckner Blvd Unit 5A, Bronx, NY
boroughs: bronx-ny, processing apartment: 2698 Bailey Ave, Bronx, NY
boroughs: bronx-ny, processing apartment: 3660 Waldo Ave Unit 8C, Bronx, NY
boroughs: bronx-ny, processing apartment: 144 Bruckner Blvd Unit 4C, Bronx, NY
boroughs: bronx-ny, processing apartment: 2816 Heath Ave Unit 06, Bronx, NY
boroughs: bronx-ny, processing apartment: 2854 Bronx Park E Unit S1, Bronx, NY
boroughs: bronx-ny, processing apartment: 3804 Greystone Ave Unit 5, Bronx, NY
boroughs: bronx-ny, processing apartment: 3254 Randall Ave Un

In [17]:
df = pd.DataFrame({'rental_title':list_rental_title,
                   'borough':list_borough,
                   'street_address':list_street_address,
                   'city':list_city,
                   'state':list_state,
                   'postal_code':list_postal_code,
                   'rating':list_rating,
                   'amenty':list_amenty,
                   'pet_policy':list_pet_policy,
                   'property_info':list_property_info,
                   'school':list_school,
                   'bedrooms':list_bedrooms,
                   'bathrooms':list_bathrooms,
                   'rent':list_rent,
                   'deposit':list_deposit,
                   'unit':list_unit,
                   'sqft':list_sqft,
                   'name':list_name,
                   'leaseLength':list_leaseLength})

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9023 entries, 0 to 9022
Data columns (total 19 columns):
amenty            9023 non-null object
bathrooms         9023 non-null object
bedrooms          9023 non-null object
borough           9023 non-null object
city              9023 non-null object
deposit           9023 non-null object
leaseLength       9023 non-null object
name              9023 non-null object
pet_policy        9023 non-null object
postal_code       9023 non-null object
property_info     9023 non-null object
rating            9023 non-null object
rent              9023 non-null object
rental_title      9023 non-null object
school            9023 non-null object
sqft              9023 non-null object
state             9023 non-null object
street_address    9023 non-null object
unit              9023 non-null object
dtypes: object(19)
memory usage: 1.3+ MB


In [19]:
df.head()

,amenty,bathrooms,bedrooms,borough,city,deposit,leaseLength,name,pet_policy,postal_code,property_info,rating,rent,rental_title,school,sqft,state,street_address,unit
0,"[Package Service, Laundry Facilities, Concierg...",1 BA,Studio,manhattan-ny,New York,,,S2,[\n\nDogs and Cats Allowed:\r\ncats and dogs P...,10025,\nProperty Information\n\n•Built in 2009\n•295...,5 star property,"$3,495 - 3,520","Avalon Morningside Park, New York, NY",[{'school_name': 'Public Elementary & Middle S...,542 Sq Ft,NY,1 Morningside Dr,
1,"[Package Service, Laundry Facilities, Concierg...",1 BA,1 BR,manhattan-ny,New York,,,A7,[\n\nDogs and Cats Allowed:\r\ncats and dogs P...,10025,\nProperty Information\n\n•Built in 2009\n•295...,5 star property,"$3,580 - 3,850","Avalon Morningside Park, New York, NY",[{'school_name': 'Public Elementary & Middle S...,746 Sq Ft,NY,1 Morningside Dr,
2,"[Package Service, Laundry Facilities, Concierg...",1 BA,1 BR,manhattan-ny,New York,,,A6,[\n\nDogs and Cats Allowed:\r\ncats and dogs P...,10025,\nProperty Information\n\n•Built in 2009\n•295...,5 star property,"$3,885","Avalon Morningside Park, New York, NY",[{'school_name': 'Public Elementary & Middle S...,748 Sq Ft,NY,1 Morningside Dr,
3,"[Package Service, Laundry Facilities, Concierg...",1 BA,1 BR,manhattan-ny,New York,,,A3,[\n\nDogs and Cats Allowed:\r\ncats and dogs P...,10025,\nProperty Information\n\n•Built in 2009\n•295...,5 star property,"$3,890","Avalon Morningside Park, New York, NY",[{'school_name': 'Public Elementary & Middle S...,718 Sq Ft,NY,1 Morningside Dr,
4,"[Package Service, Laundry Facilities, Concierg...",2 BAs,2 BRs,manhattan-ny,New York,,,B1,[\n\nDogs and Cats Allowed:\r\ncats and dogs P...,10025,\nProperty Information\n\n•Built in 2009\n•295...,5 star property,"$5,840","Avalon Morningside Park, New York, NY",[{'school_name': 'Public Elementary & Middle S...,"1,061 Sq Ft",NY,1 Morningside Dr,


In [20]:
df.to_csv('ny_rental_data.csv')